### Connecting to a MSSQL Database using ODBC

In [1]:
import pyodbc

### General connection information

NOTE: <br>
Use SQLServerManager14.msc to enable tcp, set ports and so on.

In [2]:
server = r'.\SQLEXPRESS'
port = '14430'
user = 'PythonUser'
password = 'Password1'
database = 'PythonDatabase'

### Connect to the database

In [3]:
con = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};\
                      SERVER='+server+';\
                      DATABASE='+database+';\
                      UID='+user+';\
                      PWD='+ password,
                      autocommit=True)  # Enable transactions

Create a cursor object to execute queries

In [4]:
cursor = con.cursor()

### Creating tables

We want to tables, Persons and Sales. So we check if these exist, and create them if not.

Create Persons table, if it does not exist.

In [5]:
sql_string = "SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'Persons'"

cursor.execute(sql_string)
res = cursor.fetchall()
print("Does Persons exist result: {}".format(res))

if len(res) == 0:
    persons_exists = False
else:
    if 'Persons' in res[0]:
        persons_exists = True
    else:
        persons_exists = False

if not persons_exists:
    print("Creating Persons table")
    # In the following string identity(1, 1) will enable auto increment for that row
    cursor.execute('CREATE TABLE dbo.Persons \
                        (id int not null identity(1, 1) primary key, \
                         First varchar(50) not null, \
                         Last varchar(50) not null)')
else:
    print("Persons table already exists")

Does Persons exist result: []
Creating Persons table


And create the Sales database, if it does not exist

In [6]:
sql_string = "SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'Sales'"

cursor.execute(sql_string)
res = cursor.fetchall()
print("Does Sales exist result: {}".format(res))

if len(res) == 0:
    sales_exists = False
else:
    if 'Sales' in res[0]:
        sales_exists = True
    else:
        sales_exists = False

if not sales_exists:
    print("Creating Sales table")
    cursor.execute('CREATE TABLE dbo.Sales \
                        (SaleId int not null identity(1, 1) primary key, \
                         OrderItem varchar(50) not null, \
                         OrderDescription varchar(50) null, \
                         PersonId int not null foreign key references dbo.Persons(id))')
else:
    print("Persons table already exists")

Does Sales exist result: []
Creating Sales table


### Lets create some entries in Persons database

In [7]:
def add_person(first, last):
    sql_str = "INSERT INTO [dbo].[Persons] \
                    ([First] ,[Last]) \
               VALUES \
                    ('{}', '{}')".format(first, last)
    cursor.execute(sql_str)

In [8]:
add_person('John', 'Johnson')
add_person('Søren', 'Sørensen')
add_person('Anders', 'Andersen')

Let's see what's in the database

In [9]:
cursor.execute('SELECT * FROM Persons')
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'John', 'Johnson')
(2, 'Søren', 'Sørensen')
(3, 'Anders', 'Andersen')


### And some Sales entries

In [10]:
def add_sales(item, description, person_id):
    sql_str = "INSERT INTO [dbo].[Sales] \
                   ([OrderItem],[OrderDescription],[PersonId]) \
               VALUES \
                   ('{}', '{}', {})".format(item, description, person_id)
    cursor.execute(sql_str)

In [11]:
add_sales('a thing', 'a description', 1)
add_sales('another thing', 'another description', 3)

And let's see what we got.

In [12]:
cursor.execute('SELECT * from Sales')
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'a thing', 'a description', 1)
(2, 'another thing', 'another description', 3)


Now, let's try to add a sales entry with an invalid foreign key.

In [13]:
try:
    add_sales('an invalid thing', 'i hope', 675)
except Exception as e:
    print(e)
    print("")
    print("It didn't work!")

('23000', '[23000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Sales__PersonId__245D67DE". The conflict occurred in database "PythonDatabase", table "dbo.Persons", column \'id\'. (547) (SQLExecDirectW)')

It didn't work!


It is required that the foreign key referenced exists in the other database...